In [1]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
engine = create_engine("sqlite:///../data/news.db")

In [3]:
conn = engine.connect()

---

We can check how many articles there are for each publication.

Most articles are from Reuters.

In [4]:
query = "SELECT publication, COUNT(*) AS num FROM articles GROUP BY publication ORDER BY num DESC"

results = conn.execute(text(query))

df_pub = pd.DataFrame(results.fetchall(), columns = results.keys())

In [5]:
df_pub.head()

,publication,num
0,Reuters,840094
1,The New York Times,252259
2,CNBC,238096
3,The Hill,208411
4,People,136488


---

We can check how many articles there are for each section.

There are over 900k unsorted. The majority are in the sections we would expect, e.g., world news, politics, etc.

Importantly we see that there are 2 politics sections that appear with different casing "politics" and "Politics".

In [6]:
query = "SELECT section, COUNT(*) AS num FROM articles GROUP BY section ORDER BY num DESC"

results = conn.execute(text(query))

df_sec = pd.DataFrame(results.fetchall(), columns = results.keys())

In [7]:
df_sec.head(20)

,section,num
0,,912273
1,Market News,108724
2,World News,108651
3,Business News,96395
4,Wires,67352
5,Financials,57845
6,politics,53496
7,us,51242
8,Intel,39805
9,Bonds News,39672


---

Since we are interested in the politics articles, we can check which sections contain the string "politic", this will match "politics", "political", etc.

As we saw before, we will have to check for case-insensitive matches.

Most of them are in "politics" or "Politics" as expected but there are also smaller section for different countries as well as many other sections that seem to be connected with a specific article.

In [8]:
df_sec[df_sec.section.str.contains("politic", case=False)].head(20)

,section,num
6,politics,53496
10,Politics,33875
231,World Politics,702
274,Europe Politics,438
292,Asia Politics,363
321,POLITICS,251
334,China Politics,231
386,Opinion - Politics,143
516,Political,62
1141,Political Leaders,8


---

We can check which years the articles are from.

The years range from 2016 - 2020.

In [9]:
#query = "SELECT strftime('%Y', date) AS yr, COUNT(*) AS num FROM articles GROUP BY yr"
query = "SELECT year, COUNT(*) AS num FROM articles GROUP BY year"

results = conn.execute(text(query))

df_year = pd.DataFrame(results.fetchall(), columns = results.keys())

In [10]:
df_year

,year,num
0,2016,604511
1,2017,640493
2,2018,553588
3,2019,655456
4,2020,234830


---

We can check how many articles are political articles for each year by going through the above politics sections. We can focus only on the sections with more than 10 articles.

There are only 5k politics article in 2020.

In [11]:
politics_sec = df_sec[df_sec.section.str.contains("politic", case=False)]
politics_sec.reset_index(drop=True, inplace=True)
politics_sec = politics_sec[politics_sec["num"] > 10]

politics_sec

,section,num
0,politics,53496
1,Politics,33875
2,World Politics,702
3,Europe Politics,438
4,Asia Politics,363
5,POLITICS,251
6,China Politics,231
7,Opinion - Politics,143
8,Political,62


In [12]:
df_politics_year = None

query = "SELECT year, COUNT(*) AS num FROM articles WHERE section = :sec GROUP BY year"

for s in politics_sec.section:
    results = conn.execute(text(query), {"sec": s})
    temp_df = pd.DataFrame(results.fetchall(), columns = results.keys())
    
    df_politics_year = pd.concat([df_politics_year, temp_df])

df_politics_year = df_politics_year.groupby("year").sum()

In [13]:
df_politics_year

,num
year,
2016,14906
2017,23651
2018,23607
2019,21841
2020,5556


---

In [14]:
conn.close()